# Sequence To Sequence Model - 2 Syllable Words
---

## Imports

In [1]:
from matplotlib import pyplot as plt
import random
import pickle as pkl
import pandas as pd
import numpy as np
import math
from IPython.display import clear_output

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Conv1D, MaxPooling1D, MaxPooling2D, TimeDistributed, Input, Reshape, concatenate, Conv2D, Concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

---
## Load and Visualzie the Data

In [4]:
# load data in one line
ger_train = pkl.load(open("../../saved/ger_train_syls2.pkl", "rb"))
ger_test = pkl.load(open("../../saved/ger_test_syls2.pkl", "rb"))

In [5]:
# shape of the data
print(ger_train.shape)
print(ger_test.shape)

(3312, 2)
(2928, 2)


In [6]:
ger_train.head()

contour  labels
0  [0.03177828067373946, 0.03102610675462413, 0.0...       1
1                                                 []       0
2  [0.29295377456846683, 0.23372400800342857, 0.1...       0
3  [0.3558331626873156, 0.39491268819019043, 0.41...       1
4  [0.025010218876618886, 0.024972723808391342, 0...       1

---
## Padding

In [7]:
# calculate the max length of all contours
max_length_train = max([len(contour) for contour in ger_train["contour"]])
max_length_test = max([len(contour) for contour in ger_test["contour"]])
max_length = max(max_length_train, max_length_test)

In [8]:
# pad all contours to the same length using left padding, right padding, and both side padding
def pad_contours(contours, length, padding="both"):
    padded_contours = []
    for contour in contours:
        if padding == "left":
            padded_contours.append(np.pad(contour, (length-len(contour), 0), 'constant'))
        elif padding == "right":
            padded = np.pad(contour, (0, length-len(contour)), 'constant')
            padded_contours.append(padded)
        elif padding == "both":
            padded = np.pad(contour, (math.floor((length-len(contour))/2), math.ceil((length-len(contour))/2)), 'constant')
            padded_contours.append(padded)
    return np.array(padded_contours)

In [9]:
contours_train_rp = pad_contours(ger_train["contour"], max_length, padding="right")
contours_train_lp = pad_contours(ger_train["contour"], max_length, padding="left")
contours_train_bp = pad_contours(ger_train["contour"], max_length, padding="both")

contours_test_rp = pad_contours(ger_test["contour"], max_length, padding="right")
contours_test_lp = pad_contours(ger_test["contour"], max_length, padding="left")
contours_test_bp = pad_contours(ger_test["contour"], max_length, padding="both")

In [10]:
data_train = {
    'contours': contours_train_rp.tolist(),
    'labels': ger_train['labels'].tolist(),
}
data_test = {
    'contours': contours_test_rp.tolist(),
    'labels': ger_test['labels'].tolist(),
}

ger_train_padded = pd.DataFrame(data_train)
ger_test_padded = pd.DataFrame(data_test)

ger_test_padded.head()

contours  labels
0  [0.019162689527500977, 0.018757457994703516, 0...       1
1  [0.018529939196573957, 0.01878130137324399, 0....       0
2  [0.11961875005931769, 0.121682252123691, 0.127...       0
3  [0.4515408450270049, 0.4999236321142739, 0.540...       1
4  [0.01805156620491014, 0.016649114626452348, 0....       1

---
## Reformat

In [11]:
def reformat_dataset(dataset):
    input_sequences = []  # To store sequences of feature contours
    target_sequences = []  # To store sequences of labels

    current_word_contours = []  # To accumulate contours for the current word
    current_word_labels = []    # To accumulate labels for the current word

    for i in range(len(dataset)):
        contours, label = dataset[i]

        # Check if we're starting a new word
        if i % 2 == 0:
            current_word_contours.append(contours)
            current_word_labels.append(label)
        else:
            current_word_contours.append(contours)
            current_word_labels.append(label)

            # Append the sequences for the current word
            input_sequences.append(current_word_contours)
            target_sequences.append(current_word_labels)

            # Reset for the next word
            current_word_contours = []
            current_word_labels = []

    return np.array(input_sequences), np.array(target_sequences)

In [12]:
def dataframe_to_numpy(df):
    dataset = df[['contours', 'labels']].to_numpy()
    return dataset

In [13]:
ger_train_np = dataframe_to_numpy(ger_train_padded)
ger_test_np = dataframe_to_numpy(ger_test_padded)

In [14]:
input_seqs_train, target_seqs_train = reformat_dataset(ger_train_np)
input_seqs_test, target_seqs_test = reformat_dataset(ger_test_np)
type(input_seqs_train)

numpy.ndarray

In [15]:
print(input_seqs_train.shape)
print(target_seqs_train.shape)
print()
print(input_seqs_test.shape)
print(target_seqs_test.shape)

print(target_seqs_train)

(1656, 2, 138)
(1656, 2)

(1464, 2, 138)
(1464, 2)
[[1 0]
 [0 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


---
## Model

In [16]:
num_encoder_tokens = max_length
num_decoder_tokens = 1

In [17]:
# Define the seq2seq model
input_shape = (2, 138)  # Shape of input sequences
output_shape = (2, 2)     # Shape of target sequences

encoder_inputs = Input(shape=input_shape)
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [18]:
print(state_h)
print(state_c)

KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm/PartitionedCall:2', description="created by layer 'lstm'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='lstm/PartitionedCall:3', description="created by layer 'lstm'")


In [19]:
decoder_inputs = Input(shape=output_shape)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [21]:
# Compile the model with an appropriate loss function and optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
print(input_seqs_train.shape)
print(target_seqs_train.shape)

(1656, 2, 138)
(1656, 2)


In [23]:
# Reshape the input sequences to match the model's expected input shape
input_seqs_train_rs = input_seqs_train.reshape(input_seqs_train.shape[0], input_seqs_train.shape[1], 138)

In [24]:
# Train the model using your prepared data
history = model.fit([input_seqs_train_rs, input_seqs_train_rs], target_seqs_train, epochs=10, validation_split=0.2, batch_size=16)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\abhay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\abhay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\abhay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\abhay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line

# Model 2

In [16]:
# Define input sequence length, syllable count, and feature contour size
input_seq_length = 138
syllable_count = 2
output_dim = 2  # Two classes: stressed and unstressed
latent_dim = 256  # Adjust the latent dimension as needed

In [17]:
# Define encoder inputs
encoder_inputs = Input(shape=(syllable_count, input_seq_length))

# Encoder LSTM
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# Discard encoder outputs and only keep the states
encoder_states = [state_h, state_c]

In [18]:
# Define decoder inputs
decoder_inputs = Input(shape=(None, output_dim))

# Decoder LSTM
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Dense layer for prediction
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [19]:
# Define the Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [20]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2, 138)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 2)]            0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 256),                404480    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [22]:
# One-hot encode the target labels
from keras.utils import to_categorical
num_classes = 2  # Stressed and unstressed
target_seqs_train_onehot = to_categorical(target_seqs_train, num_classes)
target_seqs_test_onehot = to_categorical(target_seqs_test, num_classes)
target_seqs_train_onehot[:2]

array([[[0., 1.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.]]], dtype=float32)

In [108]:
# Train the model
model.fit(
    [input_seqs_train, target_seqs_train_onehot],  # Input and target data
    target_seqs_train_onehot,  # Labels for the decoder (teacher forcing)
    batch_size=64,
    epochs=30,
    validation_split=0.2  # Splitting a portion of the training data for validation
)

Epoch 1/30
21/21 [==============================] - 1s 28ms/step - loss: 1.1780e-06 - accuracy: 1.0000 - val_loss: 1.2452e-06 - val_accuracy: 1.0000
Epoch 2/30
21/21 [==============================] - 1s 28ms/step - loss: 1.1689e-06 - accuracy: 1.0000 - val_loss: 1.2156e-06 - val_accuracy: 1.0000
Epoch 3/30
21/21 [==============================] - 1s 27ms/step - loss: 1.1437e-06 - accuracy: 1.0000 - val_loss: 1.2181e-06 - val_accuracy: 1.0000
Epoch 4/30
21/21 [==============================] - 1s 26ms/step - loss: 1.1392e-06 - accuracy: 1.0000 - val_loss: 1.2129e-06 - val_accuracy: 1.0000
Epoch 5/30
21/21 [==============================] - 1s 24ms/step - loss: 1.1228e-06 - accuracy: 1.0000 - val_loss: 1.1853e-06 - val_accuracy: 1.0000
Epoch 6/30
21/21 [==============================] - 0s 23ms/step - loss: 1.1155e-06 - accuracy: 1.0000 - val_loss: 1.1596e-06 - val_accuracy: 1.0000
Epoch 7/30
21/21 [==============================] - 0s 23ms/step - loss: 1.0952e-06 - accuracy: 1.0000 - v

In [109]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate([input_seqs_test, target_seqs_test_onehot], target_seqs_test_onehot)

# Print the test results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

46/46 [==============================] - 0s 4ms/step - loss: 0.0018 - accuracy: 0.9997
Test Loss: 0.0017814191523939371
Test Accuracy: 0.999658465385437


In [110]:
normal = target_seqs_test_onehot[0]
strange = target_seqs_test_onehot[1]

In [127]:
def one_hot_to_label(one_hot_encoded):
    # Convert one-hot encoded labels back to original labels
    return np.argmax(one_hot_encoded, axis=-1)


def visualize_prediction(model, encoder_input_data, decoder_input_data, target_data, index):
    # Get the input feature contours for the specified index
    encoder_input_seq = encoder_input_data[index]
    
    # Get the corresponding true labels in their original format
    true_labels = one_hot_to_label(target_data[index])
    
    # Reshape the input data to include batch dimension
    encoder_input_seq = np.expand_dims(encoder_input_seq, axis=0)
    decoder_input_seq = np.zeros((1, 2, num_classes))  # Initialize decoder input, assuming num_classes is defined
    
    # Predict using the model
    predicted_labels_one_hot = model.predict([encoder_input_seq, decoder_input_seq])
    
    # Convert predicted one-hot encoded labels back to original labels
    predicted_labels = one_hot_to_label(predicted_labels_one_hot)
    
    # Display the actual and predicted labels
    # print("Actual Labels:", true_labels)
    # print("Predicted Labels:", predicted_labels)
    x = all(predicted_labels[0][i] == true_labels[i] for i in range(len(true_labels)))
    # print("---------------")
    # print("(Correct)" if x else "(Wrong)")
    return x, true_labels, predicted_labels

In [112]:


print(normal)

normal_indices, strange_indices = [], []
for i, ele in enumerate(target_seqs_test_onehot):
    if np.array_equal(ele, normal): normal_indices.append(i)
    if np.array_equal(ele, strange): strange_indices.append(i)

print("len nomral = ", len(normal_indices))
print("len strange = ", len(strange_indices))

[[0. 1.]
 [1. 0.]]
len nomral =  1015
len strange =  446


In [113]:
target_seqs_test_onehot[1]

array([[1., 0.],
       [0., 1.]], dtype=float32)

In [114]:
class Tester:
    def __init__(self, model, A, B):
        self.model = model
        self.A = A
        self.B = B

    def check(self, i):
        x, t, p = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=i)
        print(t)
        print(p)
        print(x)

    def predict(self):
        self.A_correct = self.A_wrong = 0
        for a in self.A:
            x, _, _ = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=a)
            if x: self.A_correct += 1
            else: self.A_wrong += 1

        self.B_correct = self.B_wrong = 0
        for b in self.B:
            x, _, _ = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=b)
            if x: self.B_correct += 1
            else: self.B_wrong += 1

    def display(self):
        print("Class A")
        print("correct = ", self.A_correct)
        print("wrong = ", self.A_wrong)
        perc = (self.A_correct / (self.A_correct + self.A_wrong)) * 100
        print("Percentage = ", perc)
        
        print()

        print("Class B")
        print("correct = ", self.B_correct)
        print("wrong = ", self.B_wrong)
        perc = (self.B_correct / (self.B_correct + self.B_wrong)) * 100
        print("Percentage = ", perc)
        
        print()

        print("-------------------------")
        correct = self.A_correct + self.B_correct
        wrong = self.A_wrong + self.B_wrong
        print("correct = ", correct)
        print("wrong = ", wrong)
        perc = (correct / (correct + wrong)) * 100
        print("Percentage = ", perc)
        

In [115]:
tst = Tester(model, strange_indices, normal_indices)

In [129]:
tst.check(46)

1/1 [==============================] - 0s 71ms/step
[0 1]
[[1 1]]
False


In [117]:
tst.predict()

1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 33ms/step


In [118]:
tst.display()

Class A
correct =  147
wrong =  299
Percentage =  32.95964125560538

Class B
correct =  354
wrong =  661
Percentage =  34.87684729064039

-------------------------
correct =  501
wrong =  960
Percentage =  34.29158110882957
